In [1]:
import os
import progressbar as pb
from progressbar import ProgressBar, Percentage, Bar, ETA, FileTransferSpeed
import time
from numba import jit,autojit
import multiprocessing

import pandas as pd
import numpy as np
import warnings

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import missingno as msno
import pprint
from tabulate import tabulate


import scipy as sp
from scipy import stats



%matplotlib inline
pd.set_option("display.max_columns",80)
warnings.filterwarnings("ignore")

from IPython.display import HTML

import plotly
import cufflinks as cf
import plotly.plotly as py
import plotly.graph_objs as go
cf.go_offline()
import plotly.tools as tls
import plotly.figure_factory as ff
# tls.set_credentials_file(username='darshan.bhansali', api_key='38zZ2TS4ob4feqV0bvR0')
# tls.set_credentials_file(username='darshan-b', api_key='KuC894mmVN7kBiZZyffO')
tls.set_credentials_file(username='darshan18', api_key='dhwPNrhtYUy2iMbQqALn')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [2]:
traffic_column_names =['Recorded_Time','Received_Time','Serial_Number','Type','Subtype','Config_Version','Generated_Time',
               'Source_IP','Destination_IP','NAT_Source_IP','NAT_Destination_IP','Rule_Name','Source_User','Destination_User',
               'Application','Virtual_System','Source_Zone','Destination_Zone','Ingress_Interface','Egress_Interface',
               'Log_Forwarding_Profile','Future_Use','Session_ID','Repeat_Count','Source_Port','Destination_Port',
               'NAT_Source_Port','NAT_Destination_Port','Flags','Protocol','Action','Bytes','Bytes_sent','Bytes_received',
               'Packets','Start_Time','Elapsed_Time','Category','Future_Use_1','Sequence_Number','Action_flags','Source_Location',
               'Destination_Location', 'Future_Use_2', 'Packets Sent', 'Packets Received', 'Session End Reason','Drop_1','Drop_2',
               'Drop_3','Drop_4','Drop_5','device_name','From_Policy','a','b','c','d','e','f','g']

threat_column_names=['Recorded_Time','Received_Time','Serial_Number','Type','Subtype', 'Config_Version','Generated_Time',
                     'Source_IP','Destination_IP','NAT_Source_IP','NAT_Destination_IP','Rule_Name','Source_User','Destination_User',
                     'Application','Virtual_System','Source_Zone','Destination_Zone','Ingress_Interface','Egress_Interface', 
                     'Log_Forwarding_Profile','Future_Use','Session_ID','Repeat_Count','Source_Port','Destination_Port', 
                     'NAT_Source_Port','NAT_Destination_Port','Flags','Protocol','Action', 'Miscellaneous','Threat_ID', 
                     'Category', 'Severity', 'Direction','Sequence_Number', 'Action_Flags','Source_Location','Destination_Location',
                     'Drop_40','Drop_41','Drop_42','Drop_43','Drop_44','Something','Drop_46','Drop_47','Drop_48','Drop_49',
                     'Drop_50','Drop_51','Drop_52','Drop_53','Drop_54','Drop_55','Drop_56','Drop_57','Drop_58',
                     'device_name','Drop_60','Drop_61','Drop_62','Drop_63','Drop_64','Drop_65','Drop_66','Drop_67','Drop_68',
                     'Detailed_Subtype','Something_2','From_Policy']

In [3]:
def read_logs(rel_path,compression=None,extension=None,column_names=None):
    path = rel_path
    zip_files = [f for f in os.listdir(path) if f.endswith(extension)]
    paths = [path + x for x in zip_files]
    df=pd.DataFrame()
    for i in range(0,len(paths)):
        try:
            demo_df=pd.read_csv(paths[i],header=None,infer_datetime_format=True,low_memory=True,
                                compression=compression,error_bad_lines=False,warn_bad_lines=False,memory_map=True)
        except:
            print(i,paths[i],"\n")
            continue
        if demo_df.empty!=True:
            df=df.append(demo_df)
        else:
            print("Not empty")
    df.columns=column_names
    return df

In [4]:
root_paths=['../../data/2017-09-26/','../../data/2017-09-27/','../../data/2017-09-28/','../../data/2017-09-29/',
            '../../data/2017-09-30/','../../data/2017-10-01/','../../data/2017-10-02/','../../data/2017-10-03/',
            '../../data/2017-10-04/','../../data/2017-10-05/','../../data/2017-10-06/','../../data/2017-10-07/',
            '../../data/2017-10-08/','../../data/2017-10-09/','../../data/2017-10-10/','../../data/2017-10-11/'
           ,'../../data/2017-10-12/','../../data/2017-10-13/','../../data/2017-10-14/','../../data/2017-10-15/'
           ,'../../data/2017-10-16/']

threat_logs=pd.DataFrame()
for i in range(0,len(root_paths)):
    df2=(read_logs(root_paths[i],None,'Wireless-PA.csv',threat_column_names))
    threat_logs=pd.concat([threat_logs,df2])
threat_logs.Received_Time=pd.to_datetime(threat_logs.Received_Time)
threat_logs.sort_values(by='Received_Time',inplace=True)
threat_logs.dropna(inplace=True,subset=['Source_User'])
threat_logs.reset_index(inplace=True,drop=True)

threat_logs.drop(['Drop_40','Drop_41','Drop_42','Drop_43','Drop_44','Drop_46','Drop_47','Drop_48','Drop_49',
                 'Drop_50','Drop_51','Drop_52','Drop_53','Drop_54','Drop_55','Drop_56','Drop_57','Drop_58',
                 'Drop_60','Drop_61','Drop_62','Drop_63','Drop_64','Drop_65','Drop_65','Drop_66','Drop_67',
                 'Drop_68',
                 'NAT_Source_IP','NAT_Destination_IP','Rule_Name','Destination_User','Virtual_System',
                 'Log_Forwarding_Profile','NAT_Source_Port','NAT_Destination_Port','Action_Flags','From_Policy',
                 'Flags','Serial_Number','Config_Version','Source_Zone','Destination_Zone','Ingress_Interface',
                'Egress_Interface'],
                axis=1,inplace=True)

In [259]:
# def read_traffic():
#     traffic_logs=pd.DataFrame()
#     for i in range(0,len(root_paths)):
#         df2=(read_logs(root_paths[i],'gzip','.gz',traffic_column_names))
#         traffic_logs=pd.concat([traffic_logs,df2])
#     return traffic_logs
# traffic_logs=read_traffic()
# traffic_logs.drop(['a','b','c','d','e','f','g','From_Policy','Drop_1','Drop_2','Drop_3','Drop_4','Drop_5',
#                    'Future_Use_2','Action_flags','Future_Use_1','NAT_Source_Port','NAT_Destination_Port',
#                ""    'NAT_Source_IP','NAT_Destination_IP','Rule_Name','Destination_User','Virtual_System',
#                    'Log_Forwarding_Profile','Serial_Number','Config_Version','Source_Zone','Destination_Zone',
#                    'Ingress_Interface','Egress_Interface'],axis=1,inplace=True)
# traffic_logs.Received_Time=pd.to_datetime(traffic_logs.Received_Time,errors='coerce')
# traffic_logs.sort_values(by='Received_Time',inplace=True)
# traffic_logs.dropna(inplace=True,subset=['Source_User'])
# traffic_logs.reset_index(inplace=True,drop=True)

In [258]:
print("The period for which the analysis has been performed is from {} to {}".format(threat_logs.Received_Time.min(),
                                                                                    threat_logs.Received_Time.max()))

The period for which the analysis has been performed is from 2017-09-26 19:59:22 to 2017-10-16 23:59:46


### 1.  Histogram of #incidents/#users (bins for both)


In [261]:
d=pd.DataFrame(threat_logs.Source_User.value_counts())
d.reset_index(inplace=True,drop=False)
d.columns=['Source_User','Count']
d_le1=d[d.Count==1]
d_ge1=d[d.Count>1]

In [262]:
trace = go.Histogram(x=d.Count)

layout=go.Layout(title='Histogram of # Incidents/User',
                 xaxis=dict(title='Count of repeat incidents',tickangle=0),
                 yaxis=dict(title='Number of Users'))
data=[trace]
fig = go.Figure(data=data,layout=layout)
py.iplot(fig, filename='Histogram_Full')

In [254]:
print("Only {} users  have more than 20 threat incidents\nOnly {} users have more than 50 threat incidents\nOnly {} users have more than 100 threat incidents"
      .format(sum(d.Count>20),sum(d.Count>50),sum(d.Count>100)))

Only 26 users  have more than 20 threat incidents
Only 15 users have more than 50 threat incidents
Only 6 users have more than 100 threat incidents


In [263]:
print("Lets plot the histogram for users with less than 20 incidents")
trace = go.Histogram(x=d.Count,xbins=dict(start=0,end=20))

layout=go.Layout(title='Histogram of # Incidents/User',
                 xaxis=dict(title='Count of repeat incidents',tickangle=0),
                 yaxis=dict(title='Number of Users'))
data=[trace]
fig = go.Figure(data=data,layout=layout)
py.iplot(fig, filename='Histogram_20')

Lets plot the histogram for users with less than 20 incidents


In [265]:
print("Lets plot the histogram for users with less than 50 incidents")
trace = go.Histogram(x=d.Count,xbins=dict(start=0,end=50))

layout=go.Layout(title='Histogram of # Incidents/User',
                 xaxis=dict(title='Count of repeat incidents',tickangle=0),
                 yaxis=dict(title='Number of Users'))
data=[trace]
fig = go.Figure(data=data,layout=layout)
py.iplot(fig, filename='Histogram_50')

Lets plot the histogram for users with less than 50 incidents


In [266]:
print("Lets plot the histogram for users with less than 100 incidents")
trace = go.Histogram(x=d.Count,xbins=dict(start=0,end=100))

layout=go.Layout(title='Histogram of # Incidents/User',
                 xaxis=dict(title='Count of repeat incidents',tickangle=0),
                 yaxis=dict(title='Number of Users'))
data=[trace]
fig = go.Figure(data=data,layout=layout)
py.iplot(fig, filename='Histogram_100')

Lets plot the histogram for users with less than 100 incidents


### 2. For users with 1 incident, number medium, high, critical


In [121]:
d=threat_logs.groupby([threat_logs.Source_User,threat_logs.Severity]).count().unstack().fillna(0)['Subtype']
c=pd.DataFrame(d)
c.reset_index(inplace=True,drop=False)
c['Count']=c.critical+c.high+c.medium
c.sort_values(by='Count',ascending=False,inplace=True)
c.Count=c.Count.astype(int)
c.reset_index(inplace=True,drop=True)
x=c[c.Count<=1]
y=c[c.Count>1]

In [122]:
print("The table below illustrates the number of incidents for Users with 1 incident in the threat logs by Severity")
print("\n-----------------------------------------------------------------------------------------------------------\n")
print(tabulate(x[['Source_User','critical','high','medium']],headers=['Source User','Critical','High','Medium'],tablefmt='grid'))

The table below illustrates the number of incidents for Users with 1 incident in the threat logs by Severity

-----------------------------------------------------------------------------------------------------------

+-----+------------------------------------------------------------------+------------+--------+----------+
|     | Source User                                                      |   Critical |   High |   Medium |
+=====+==================================================================+============+========+==========+
| 393 | a56e530cb71d921e2a06cb8a11f05497dbb11ad47f38f2e2642c50c8ee7732c7 |          0 |      0 |        1 |
+-----+------------------------------------------------------------------+------------+--------+----------+
| 394 | b4abf347596882503b36d7e3f0d69ca7de239f191e3214321351d14d982ef499 |          0 |      0 |        1 |
+-----+------------------------------------------------------------------+------------+--------+----------+
| 395 | aaca5cdcccf23e9d5

### 3.a. For users with 1 incident, number for each type of incident

In [133]:
d=threat_logs.groupby([threat_logs.Source_User,threat_logs.Subtype]).count().unstack().fillna(0)['Severity']
c=pd.DataFrame(d)
c.reset_index(inplace=True,drop=False)
c['Count']=c.spyware+c.vulnerability
c.sort_values(by='Count',ascending=False,inplace=True)
c.Count=c.Count.astype(int)
c.reset_index(inplace=True,drop=True)
x=c[c.Count<=1]
y=c[c.Count>1]

In [134]:
print("The table below illustrates the number of incidents for Users with 1 incident in the threat logs by Incident")
print("\n---------------------------------------------------------------------------------------------------------------------\n")
print(tabulate(x[['Source_User','spyware','vulnerability']],headers=['Source_User','spyware','vulnerability'],tablefmt='grid'))

The table below illustrates the number of incidents for Users with 1 incident in the threat logs by Incident

---------------------------------------------------------------------------------------------------------------------

+-----+------------------------------------------------------------------+-----------+-----------------+
|     | Source_User                                                      |   spyware |   vulnerability |
+=====+==================================================================+===========+=================+
| 393 | ef03d74e4e072361f2c6a9a48d3ff04218c45b5f144b40d6bc46a285a3d9f535 |         1 |               0 |
+-----+------------------------------------------------------------------+-----------+-----------------+
| 394 | ee15050146c22a4b080c0518abcd868d38809b6ac450750698fadec3f2be2287 |         1 |               0 |
+-----+------------------------------------------------------------------+-----------+-----------------+
| 395 | ee5d3aa446b8deda0193b3c49001

### 3.b. For users with 1 incident, number for each type of Detailed Subtype

In [157]:
d=threat_logs.groupby([threat_logs.Source_User,threat_logs.Detailed_Subtype]).count().unstack().fillna(0)['Severity']
c=pd.DataFrame(d)
c.reset_index(inplace=True,drop=False)
c['Count']=c[c.columns[1:]].sum(axis=1)
c.sort_values(by='Count',ascending=False,inplace=True)
c.Count=c.Count.astype(int)
c.reset_index(inplace=True,drop=True)
x=c[c.Count<=1]
y=c[c.Count>1]

In [175]:
print("The table below illustrates the number of incidents for Users with 1 incident in the threat logs by Detailed Subtype")
print("\n---------------------------------------------------------------------------------------------------------------------\n")
x.iloc[:,:-1]

The table below illustrates the number of incidents for Users with 1 incident in the threat logs by Detailed Subtype

---------------------------------------------------------------------------------------------------------------------



Detailed_Subtype,Source_User,brute-force,code-execution,dns,dns-wildfire,dos,info-leak,overflow,spyware,sql-injection
393,a56e530cb71d921e2a06cb8a11f05497dbb11ad47f38f2...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
394,b4abf347596882503b36d7e3f0d69ca7de239f191e3214...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
395,aaca5cdcccf23e9d5f1debe1ec0528fc5550ffe99f2f4a...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
396,aae33f99f3b8e226134b0a388e1392e2fa07e0b7371112...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
397,ad83c973582e61a5a8f835290abbba7905c6419dc3f2c0...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
398,adedc265e5231a92104bc66a625c155fc15445ea36312d...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
399,aaffb0f4e1aaaf8f63d821569e1404bdce5859fd752d62...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
400,ae67d5bfe67edc587454affa2ea45a82225be653837c66...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
401,ad93b36a16702743ed28b944b13c74840d77840b65389f...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
402,b51e7774251c7abac2116b2c7361bc672f60df7e879460...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### 4. For users with more than 1 incident, percentage of medium, high, critical

In [203]:
d=threat_logs.groupby([threat_logs.Source_User,threat_logs.Severity]).count().unstack().fillna(0)['Subtype']
c=pd.DataFrame(d)
c.reset_index(inplace=True,drop=False)
c['Count']=c.critical+c.high+c.medium
c.sort_values(by='Count',ascending=False,inplace=True)
c.Count=c.Count.astype(int)
c.reset_index(inplace=True,drop=True)
x=c[c.Count<=1]
y=c[c.Count>1]

In [204]:
y.critical=y.critical/len(threat_logs[threat_logs.Severity=='critical'])
y.high=y.high/len(threat_logs[threat_logs.Severity=='high'])
y.medium=y.medium/len(threat_logs[threat_logs.Severity=='medium'])
print("The table below illustrates the percetange of incidents for Users with more than 1 incident in the threat logs by Severity")
print("\n-----------------------------------------------------------------------------------------------------------\n")
print(tabulate(y[['Source_User','critical','high','medium']],headers=['Source User','Critical %','High %','Medium %'],tablefmt='grid'))

The table below illustrates the percetange of incidents for Users with more than 1 incident in the threat logs by Severity

-----------------------------------------------------------------------------------------------------------

+-----+------------------------------------------------------------------+--------------+------------+-------------+
|     | Source User                                                      |   Critical % |     High % |    Medium % |
+=====+==================================================================+==============+============+=============+
|   0 | e0027ef4773a01094339852b6b1207f38c22adbe46879a9faf490413b580caab |    0.0169492 | 0.0358025  | 0.149456    |
+-----+------------------------------------------------------------------+--------------+------------+-------------+
|   1 | 07a805db92f77828144af02c110b4107381c8608ff69cbe8322b1619550dca26 |    0         | 0.825926   | 0           |
+-----+----------------------------------------------------------

### 5.a. For users with more than 1 incident, percentage of type of incident

In [220]:
d=threat_logs.groupby([threat_logs.Source_User,threat_logs.Subtype]).count().unstack().fillna(0)['Severity']
c=pd.DataFrame(d)
c.reset_index(inplace=True,drop=False)
c['Count']=c[c.columns[1:]].sum(axis=1)
c.sort_values(by='Count',ascending=False,inplace=True)
c.Count=c.Count.astype(int)
c.reset_index(inplace=True,drop=True)
y=c[c.Count>1]

In [221]:
y.spyware=y.spyware/len(threat_logs[threat_logs.Subtype=='spyware'])
y.vulnerability=y.vulnerability/len(threat_logs[threat_logs.Subtype=='vulnerability'])
print("The table below illustrates the percetange of incidents for Users with more than 1 incident in the threat logs by Incident")
print("\n-----------------------------------------------------------------------------------------------------------\n")
print(tabulate(y[['Source_User','spyware','vulnerability']],headers=['Source User','Spyware %','Vulnerability %'],tablefmt='grid'))

The table below illustrates the percetange of incidents for Users with more than 1 incident in the threat logs by Incident

-----------------------------------------------------------------------------------------------------------

+-----+------------------------------------------------------------------+-------------+-------------------+
|     | Source User                                                      |   Spyware % |   Vulnerability % |
+=====+==================================================================+=============+===================+
|   0 | e0027ef4773a01094339852b6b1207f38c22adbe46879a9faf490413b580caab | 0.200256    |       0.0140271   |
+-----+------------------------------------------------------------------+-------------+-------------------+
|   1 | 07a805db92f77828144af02c110b4107381c8608ff69cbe8322b1619550dca26 | 0           |       0.302715    |
+-----+------------------------------------------------------------------+-------------+-------------------+
|   

### 5.b. For users with more than 1 incident, percentage of type of incident

In [252]:
d=threat_logs.groupby([threat_logs.Source_User,threat_logs.Detailed_Subtype]).count().unstack().fillna(0)['Severity']
c=pd.DataFrame(d)
c.reset_index(inplace=True,drop=False)
c['Count']=c[c.columns[1:]].sum(axis=1)
c.sort_values(by='Count',ascending=False,inplace=True)
c.Count=c.Count.astype(int)
c.reset_index(inplace=True,drop=True)
y=c[c.Count>1]

In [253]:
b=y.columns.get_values()[:-1]
b=b[1:]
for i in range(0,len(b)):
    y[b[i]]=y[b[i]]/len(threat_logs[threat_logs.Detailed_Subtype==b[i]])
print("Table below illustrates the percetange of incidents for Users with more than 1 incident in the threat logs by Detailed Subtype")
print("\n-------------------------------------------------------------------------------------------------------------------------------\n")
y.reset_index(drop=True,inplace=True)
y.iloc[:,:-1]

Table below illustrates the percetange of incidents for Users with more than 1 incident in the threat logs by Detailed Subtype

-------------------------------------------------------------------------------------------------------------------------------



Detailed_Subtype,Source_User,brute-force,code-execution,dns,dns-wildfire,dos,info-leak,overflow,spyware,sql-injection
0,e0027ef4773a01094339852b6b1207f38c22adbe46879a...,0.037396,0.055556,0.226894,0.199609,0.001773,0.013699,0.000000,0.000000,0.000000
1,07a805db92f77828144af02c110b4107381c8608ff69cb...,0.925208,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,eb30caaced9156871a8c10b8f96924d1dd57bc3bfbcbf0...,0.000000,0.000000,0.000000,0.000000,0.332447,0.000000,0.000000,0.000000,0.000000
3,e931f318e6d489b8c1fefcdbcc9f1678c3f5fc95b51fa9...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.832061,0.000000
4,9da2d1a02cd95e57b98d0f8e631ad2fcf7d0b8c5ff94a5...,0.000000,0.000000,0.000000,0.000000,0.271277,0.000000,0.000000,0.000000,0.000000
5,1d87fa38d0fe0ba141f715757c6ae4ef2b44d114c702c7...,0.000000,0.000000,0.000000,0.000000,0.172872,0.000000,0.000000,0.000000,0.000000
6,271be6dd221205698a91c021260cf5bf4deb604945a2e8...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.386454,0.000000,0.000000
7,eee73a52c55dbef6c5b4ec7ec0cd3d200a90f528b7344b...,0.000000,0.833333,0.000000,0.000000,0.000000,0.876712,0.000000,0.000000,0.857143
8,64c7c7ab1002f853fec50dcb65b8d4ce104aa67df24a11...,0.000000,0.000000,0.028695,0.007828,0.000000,0.000000,0.000000,0.000000,0.000000
9,6b0394c114737385c681063f719ccbd4217d866a8984ab...,0.000000,0.000000,0.000000,0.000000,0.078014,0.000000,0.000000,0.000000,0.000000


In [260]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To toggle code, click <a href="javascript:code_toggle()">here</a>.''')